In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd

baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")

/Users/daphnehansell/miniconda3/envs/policyengine/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
reform = Reform.from_dict({
  "gov.aca.ptc_phase_out_rate[0].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[1].amount": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[2].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.ptc_phase_out_rate[3].amount": {
    "2026-01-01.2100-12-31": 0.02
  },
  "gov.aca.ptc_phase_out_rate[4].amount": {
    "2026-01-01.2100-12-31": 0.04
  },
  "gov.aca.ptc_phase_out_rate[5].amount": {
    "2026-01-01.2100-12-31": 0.06
  },
  "gov.aca.ptc_phase_out_rate[6].amount": {
    "2026-01-01.2100-12-31": 0.085
  },
  "gov.aca.ptc_income_eligibility[2].amount": {
    "2026-01-01.2100-12-31": True
  }
}, country_id="us")




In [3]:
baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
reformed = Microsimulation(reform=reform, dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
weights = baseline.calculate("household_weight", period=2024)



In [4]:
baseline_aca_eligible = baseline.calculate("is_aca_ptc_eligible", map_to="tax_unit", period=2026).sum()
baseline_aca_eligible/1e6

34.4898908313967

In [5]:
baseline_aca_enrollment = baseline.calculate("takes_up_aca_if_eligible", map_to="person", period=2026).sum()
baseline_aca_enrollment/1e6

228.37424343962823

In [6]:
period = 2025
sim    = baseline

# ── Tax-unit flags, broadcast to people ──────────────────────────────────────
takes_up = sim.calculate("takes_up_aca_if_eligible",
                         map_to="person", period=period)        # 0/1
aca_ptc  = sim.calculate("aca_ptc",
                         map_to="person", period=period)        # $ amount

# ── PERSON weights (pick any person-level variable) ─────────────────────────
person_wt = sim.calculate("age", map_to="person", period=period).weights

# ── Build mask & sum weights ────────────────────────────────────────────────
mask = (takes_up == 1) & (aca_ptc > 0)

people_with_ptc_takeup_wtd = (mask.astype(float) * person_wt).sum()

print(f"{people_with_ptc_takeup_wtd:,.0f} weighted people live in tax units "
      "that take up Marketplace coverage and actually receive a PTC.")


191,573,928 weighted people live in tax units that take up Marketplace coverage and actually receive a PTC.


In [7]:
period = 2026

# ── Tax-unit flags, broadcast to people ──────────────────────────────────────
takes_up_r = reformed.calculate("takes_up_aca_if_eligible",
                         map_to="person", period=period)        # 0/1
aca_ptc_r  = reformed.calculate("aca_ptc",
                         map_to="person", period=period)        # $ amount

# ── PERSON weights (pick any person-level variable) ─────────────────────────
person_wt_r = reformed.calculate("age", map_to="person", period=period).weights

# ── Build mask & sum weights ────────────────────────────────────────────────
mask = (takes_up_r == 1) & (aca_ptc_r > 0)

people_with_ptc_takeup_wtd_r = (mask.astype(float) * person_wt_r).sum()

print(f"{people_with_ptc_takeup_wtd_r:,.0f} weighted people live in tax units "
      "that take up Marketplace coverage and actually receive a PTC.")


24,630,096 weighted people live in tax units that take up Marketplace coverage and actually receive a PTC.


In [8]:
period = 2026
sim    = baseline

# ── Tax-unit flags, broadcast to people ──────────────────────────────────────
takes_up = sim.calculate("takes_up_aca_if_eligible",
                         map_to="person", period=period)        # 0/1
aca_ptc  = sim.calculate("aca_ptc",
                         map_to="person", period=period)        # $ amount

# ── PERSON weights (pick any person-level variable) ─────────────────────────
person_wt = sim.calculate("age", map_to="person", period=period).weights

# ── Build mask & sum weights ────────────────────────────────────────────────
mask = (takes_up == 1) & (aca_ptc > 0)

people_with_ptc_takeup_wtd = (mask.astype(float) * person_wt).sum()

print(f"{people_with_ptc_takeup_wtd:,.0f} weighted people live in tax units "
      "that take up Marketplace coverage and actually receive a PTC.")


20,074,006 weighted people live in tax units that take up Marketplace coverage and actually receive a PTC.


In [9]:
year = 2026
state = baseline.calculate("state_code", map_to="household", period=year)
num_dependents = baseline.calculate("tax_unit_dependents", map_to="household", period=year)
married = baseline.calculate("is_married", map_to="household", period=year)
employment_income = baseline.calculate("employment_income", map_to="household", period=year)
self_employment_income = baseline.calculate("self_employment_income", map_to="household", period=year)
aca_baseline = baseline.calculate("aca_ptc", map_to="household", period=year)
rating_area = baseline.calculate("slcsp_rating_area", map_to="household", period=year)
household_id = baseline.calculate("household_id", map_to="household", period=year)
aca_reform = reformed.calculate("aca_ptc", map_to="household", period=year)

In [10]:
# Create a DataFrame with the outputs
data = {
    "household_id": household_id,
    "State": state,
    "Married": married,
    "Num_Dependents": num_dependents,
    "Employment_Income": employment_income,
    "aca_baseline": aca_baseline,
    "aca_reform": aca_reform,

   }


df_outputs = pd.DataFrame(data)
df_outputs[df_outputs['household_id'] == 4428]


,household_id,State,Married,Num_Dependents,Employment_Income,aca_baseline,aca_reform
600,4428,MA,1.0,4.0,52859.65625,0.0,0.0


In [11]:
# -------------------------------------------------------------
# 0️⃣  Make sure the CPS household weight is in the DataFrame
# -------------------------------------------------------------
# If you already stuffed it in earlier, skip this.
df_outputs["weight"] = aca_baseline.weights    # aligns by household_id

# -------------------------------------------------------------
# 1️⃣  Define a weight threshold for “reasonably representative”
# -------------------------------------------------------------
MIN_WT = 10_000          # ↖ try 5_000 if you want a looser cut

df_big = df_outputs[df_outputs["weight"] >= MIN_WT].copy()

# -------------------------------------------------------------
# 2️⃣  Net PTC change and (optionally) weighted national impact
# -------------------------------------------------------------
df_big["net_change"] = df_big["aca_reform"] - df_big["aca_baseline"]
df_big["wt_change"]  = df_big["net_change"] * df_big["weight"]  # national $ impact

# -------------------------------------------------------------
# 3️⃣  Biggest ↑ increases and ↓ decreases, LIMITED to big-weight HHs
# -------------------------------------------------------------
N = 10   # how many households to show in each direction

cols = ["household_id", "State", "weight", "net_change", "wt_change"]

top_increases = df_big.nlargest(N, "net_change")[cols]
top_decreases = df_big.nsmallest(N, "net_change")[cols]

print("Most positive net-income changes (PTC boosts):")
display(top_increases)

print("\nMost negative net-income changes (PTC cuts):")
display(top_decreases)


Most positive net-income changes (PTC boosts):


,household_id,State,weight,net_change,wt_change
15170,63406,TX,96633.976562,20097.220703,1.942074e+09
11278,46357,FL,63879.199219,15559.541992,9.939311e+08
30957,132558,GA,34982.687500,14077.200195,4.924583e+08
14628,61712,TX,15532.655273,14010.416992,2.176190e+08
11156,46034,FL,30328.830078,9943.345703,3.015700e+08
31402,133962,FL,21518.343750,9887.317383,2.127587e+08
4013,16074,OH,11105.104492,6990.998047,7.763576e+07
31632,134802,FL,23929.857422,5788.573242,1.385197e+08
9387,39878,NC,23245.363281,5382.159180,1.251102e+08
9643,40712,SC,13224.298828,4779.596680,6.320681e+07



Most negative net-income changes (PTC cuts):


,household_id,State,weight,net_change,wt_change
9,78,ME,20615.609375,0.0,0.0
25,194,ME,14024.491211,0.0,0.0
26,199,ME,17028.466797,0.0,0.0
38,326,ME,14531.310547,0.0,0.0
44,356,ME,15670.582031,0.0,0.0
47,380,ME,20000.724609,0.0,0.0
48,407,ME,10724.878906,0.0,0.0
51,425,ME,15117.451172,0.0,0.0
53,437,ME,11405.429688,0.0,0.0
58,475,ME,12578.248047,0.0,0.0


In [12]:
df_outputs[df_outputs['household_id'] == 4428]


,household_id,State,Married,Num_Dependents,Employment_Income,aca_baseline,aca_reform,weight
600,4428,MA,1.0,4.0,52859.65625,0.0,0.0,23814.501953


In [13]:
# 0. Make sure net_change exists
df_outputs["net_change"] = df_outputs["aca_reform"] - df_outputs["aca_baseline"]

# 1. Flag households with any change
mask = df_outputs["net_change"] != 0          # True for ↑ or ↓

# 2. Weighted mean among those households
avg_net_change = (
    (df_outputs.loc[mask, "net_change"] * df_outputs.loc[mask, "weight"]).sum()
    / df_outputs.loc[mask, "weight"].sum()
)

print(f"Average weighted PTC change among households with any change: "
      f"${avg_net_change:,.2f}")


Average weighted PTC change among households with any change: $2,239.40


In [14]:
# ------------------------------------------------------------------
# 0.  Ensure supporting columns exist
# ------------------------------------------------------------------
df_outputs["net_change"] = df_outputs["aca_reform"] - df_outputs["aca_baseline"]

# ------------------------------------------------------------------
# 1.  Keep only households with a PTC in *both* scenarios
# ------------------------------------------------------------------
mask_both_ptc = (df_outputs["aca_baseline"] > 0) & (df_outputs["aca_reform"] > 0)
df_dual_ptc   = df_outputs[mask_both_ptc]

# ------------------------------------------------------------------
# 2.  Weighted average of the net change (household perspective)
# ------------------------------------------------------------------
avg_net_change_dual_hh = (
    (df_dual_ptc["net_change"] * df_dual_ptc["weight"]).sum()
    / df_dual_ptc["weight"].sum()
)

print(f"Average weighted PTC change among households with a PTC in both "
      f"baseline and reform: ${avg_net_change_dual_hh:,.2f}")


Average weighted PTC change among households with a PTC in both baseline and reform: $1,784.16


In [15]:
# ------------------------------------------------------------------
# 0.  Ensure supporting columns exist (already done above)
# ------------------------------------------------------------------
df_outputs["net_change"] = df_outputs["aca_reform"] - df_outputs["aca_baseline"]

# ------------------------------------------------------------------
# 1.  Keep only households that *gain* a PTC (reform > 0, baseline == 0)
# ------------------------------------------------------------------
mask_reform_only = (df_outputs["aca_baseline"] == 0) & (df_outputs["aca_reform"] > 0)
df_reform_only   = df_outputs[mask_reform_only]

# ------------------------------------------------------------------
# 2.  Weighted average of the net change (household perspective)
# ------------------------------------------------------------------
avg_net_change_reform_only_hh = (
    (df_reform_only["net_change"] * df_reform_only["weight"]).sum()
    / df_reform_only["weight"].sum()
)

print(f"Average weighted PTC change among households that newly receive a PTC "
      f"under the reform: ${avg_net_change_reform_only_hh:,.2f}")


Average weighted PTC change among households that newly receive a PTC under the reform: $4,797.60


In [16]:
import numpy as np
from policyengine_us import Simulation

# -------------------------------
# 1. Pull household-level results
# -------------------------------
# ACA PTC (baseline and reform)
ptc_base   = baseline.calculate("aca_ptc", map_to="household", period=2026)
ptc_reform = reformed.calculate("aca_ptc",   map_to="household", period=2026)

# Household weights (same for both sims)
hh_wt      = baseline.calculate("household_weight", map_to="household", period=2026)

# -------------------------------
# 2. Weighted sum of the change
# -------------------------------
weighted_total_change = ptc_reform - ptc_base

# Optional: average change per household
weighted_total_change.sum()/1e9

29.856163170330728

In [17]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# ------------------------------------------------------------------
# Brand hex codes (one-to-one with style.colors)
# ------------------------------------------------------------------
COLOR_BLUE               = "#2C6496"   # style.colors.BLUE  / BLUE_PRIMARY
COLOR_BLUE_LIGHT         = "#D8E6F3"   # style.colors.BLUE_LIGHT / BLUE_95
COLOR_LIGHT_GRAY         = "#F2F2F2"   # style.colors.LIGHT_GRAY
COLOR_MEDIUM_LIGHT_GRAY  = "#BDBDBD"   # style.colors.MEDIUM_LIGHT_GRAY
COLOR_DARK_GRAY          = "#616161"   # style.colors.DARK_GRAY

# ––– choose colours for positive vs. negative average bars –––
POS_COLOR = COLOR_BLUE
NEG_COLOR = COLOR_DARK_GRAY

# ------------------------------------------------------------------
# 1.  Pull baseline / reform net income + weights
# ------------------------------------------------------------------
net_base   = baseline.calculate(
    "household_net_income_including_health_benefits", map_to="household", period=2026
)
net_reform = reformed.calculate(
    "household_net_income_including_health_benefits", map_to="household", period=2026
)
weights    = baseline.calculate(
    "household_weight", map_to="household", period=2026
)

df = pd.DataFrame({
    "net_base": net_base,
    "delta":    net_reform - net_base,
    "weight":   weights,
})

# ------------------------------------------------------------------
# 2.  Weighted decile edges (baseline ranking)
# ------------------------------------------------------------------
def wquantile(values, qs, w):
    srt = np.argsort(values)
    values, w = values[srt], w[srt]
    cum_w = np.cumsum(w) / np.sum(w)
    return np.interp(qs, cum_w, values)

edges = wquantile(df["net_base"].values,
                  np.linspace(0, 1, 11), df["weight"].values)

df["decile"] = pd.cut(df["net_base"],
                      bins=edges,
                      labels=np.arange(1, 11),
                      include_lowest=True)

# ------------------------------------------------------------------
# 3.  Weighted average Δnet-income by decile
# ------------------------------------------------------------------
decile_avg = (
    df.groupby("decile")
      .apply(lambda g: np.average(g["delta"], weights=g["weight"]))
      .reset_index(name="avg_change")
)

# ------------------------------------------------------------------
# 4.  Use brand colours: blue if gain, dark-gray if loss
# ------------------------------------------------------------------
bar_colors = [
    POS_COLOR if v >= 0 else NEG_COLOR
    for v in decile_avg["avg_change"]
]

# ------------------------------------------------------------------
# 5.  Plot
# ------------------------------------------------------------------
fig = go.Figure(
    data=[
        go.Bar(
            x=decile_avg["decile"].astype(int),
            y=decile_avg["avg_change"],
            marker_color=bar_colors,
            text=decile_avg["avg_change"].apply(lambda v: f"${v:,.0f}"),
            textposition="inside",
        )
    ],
    layout=dict(
        title="Impact of Extending IRA PTC Expansion by Income Decile – 2026",
        xaxis_title="Income Decile",
        yaxis_title="Average change in household net income ($)",
        showlegend=False,
    )
)
fig.add_hline(y=0, line_width=1, line_color="black")
fig.show()
fig.update_xaxes(dtick=1)        # show 1-10 instead of only the evens
